# The Aarseth NBODY family of codes (*draft*)

With the 
[passing of Sverre Aarseth on Dec 28, 2024](https://ascl.net/wordpress/2025/01/07/sverre-aarseth-father-of-open-source-stellar-dynamics-has-passed-on-to-a-higher-orbit/),
we remember him by showcasing some examples of some of his legacy codes in NEMO.

Sverre has always made his code available for anybody to use. His current body of work can still be found at https://people.ast.cam.ac.uk/~sverre/web/pages/nbody.htm  as well as an [entry in ASCL](https://ascl.net/1102.006).


The following programs are avaiilable in NEMO:

* [nbody0, nbody00](https://teuben.github.io/nemo/man_html/nbody0.1.html) - version from Binney & Tremaine's "Galactic Dynamics" (1987) book, dubbed the Micky Mouse version by Sverre
* nbody1, runbody1
* nbody2, runbody2
* nbody4, runbody4
* nbody5 - Regularized AC N-body code with triple & binary collisions
* nbody6, runbody6 - Hermite N-body code with optional stellar evolution
* firstn -  von Hoerners first N-body code
* hermit
* u4tos, stou4 - conversion programs of Sverre's "UNIT4" files to and from NEMO snapshot's
* u3tos - conversion program of Sverre's "UNIT3" file to a NEMO snapshot

and Sverre's 1999 paper ["From NBODY1 to NBODY6: The Growth of an Industry"](https://ui.adsabs.harvard.edu/abs/1999PASP..111.1333A) 
outlines the history behind this series.



# Loading NEMO

We start by loading NEMO in the shell (your location will likely differ)

In [ ]:
source $HOME/NEMO/nemo/nemo_start.sh
nemo

# nbody0, nbody00, nbody0_ff, nbody0h4

This code was published in the Appendix of the 1987 (first) edition of Binney & Tremaine's *Galactic Dynamics*. The code
can be found in 
**$NEMO/src/nbody/evolve/aarseth/nbody0**, where several derivatives of this *Micky Mouse* version are available.

In [ ]:
man nbody0

### Creating initial conditions

By default the FORTRAN code is compiled with space for a maximum of 2048 particles. We thus create a Plummer (1911) sphere with 2048 particles. We fix the seed to have reproducable results, and integrate a few crossing times to keep the CPU loaded for a few seconds.

In [ ]:
rm -f p2048
mkplummer p2048 2048 seed=123
tsf p2048

### Comparing FORTRAN and C

We now compare the performance of the FORTRAN and C versions. We use **out=.** to not have to write an output file, saving some overhead.
The default integration time **tcrit=2** is used.

In [ ]:
/usr/bin/time nbody0 p2048 . tcrit=2
/usr/bin/time nbody00 p2048 . tcrit=2

### Reproducability

If NEMO's random number generator is working correctly, the number of steps and energy at time=2 should be identical to
```
   time = 2   steps = 161238   energy = -0.244397
```
and although the CPU time varies per machine, my 2023 "Ultra 7 155H" laptop CPU took about 1.7sec for **nbody0** and 2.0sec for **nbody00**. Also notable is that the C version does use a small amount (4%) of system time, whereas FORTRAN took 0. 

### nbody00_ff

The pure FORTRAN version does not have a NEMO CLI. It reads a one line header from stdin, followed by N (the number of bodies) lines containing the mass, position and velocity. The header contains **n,eta,deltat,tcrit,eps2,reset**.   Such an input file can be easily created using basic NEMO tools.

In [ ]:
rm -f p5
mkplummer p5 5 seed=123

# convert the snapsshot to the input file that nbody0_ff needs
echo "5,0.02,1.0,10,0.0001,1" > input5
snapprint p5 m,x,y,z,vx,vy,vz format=%.15g >> input5

# run nbody0_ff
nbody0_ff < input5 

# run nbody0, and compare the phase space coordinated at times=10
nbody0 p5 - deltat=1 eps=0.01 tcrit=10 | snaptrim - - times=10 | snapprint -

### Comparing nbody0 and nbody0_ff

Apart from the limited accuracy that nbody0_ff shows, the comparison is excellent, as well as number of steps taken and the energy in the final snapshot:
```
     time = 10   steps = 2351   energy = -0.181269
```

# nbody1, runbody1

This code is an official version from 1997, though minor updates have been tracked up to a 2019 fix to the KZ(15) parameter. Within NEMO the code lives in **$NEMO/src/nbody/evolve/aarseth/nbody1**

This version also introduces the more formal *run* interface in NEMO to allow one to run legacy codes with a NEMO command line interface. It also includes conversion between NEMO's snapshots and NBODYx files (unit3, unit4).

# nbody2, runbody2


## A Plummer sphere

To create a Plummer (1911) sphere with 128 particles the **mkplummer** program is available.   



In [ ]:
rm -f p128 
mkplummer p128 128

The contents is binary.  So viewing contents needs a program **tsf** (*type structured file*)

In [ ]:
tsf p128

## Integrate it:

The **hackcode1** program is the original Barnes and Hut (1985) tree code N-logN algorithm to integrate an N-body system.

In [ ]:
hackcode1 p128 p128.out tstop=2 

Display some diagnostics, such as energy and center of mass motion

In [ ]:
snapdiagplot p128.out yapp=fig1.png/png
ls


This gives around 1% energy conservation, whichis not too bad, at least for galactic type simulations. For star clusters probably not too great at all.

![alt text](fig1.png "Title")

Now plot in a two by two panel the X-Y projections of the integration at times 0, 1 and 2.

In [ ]:
snapplot p128.out nxy=2,2 times=0,1,2 yapp=fig2.png/png
ls

![alt text](fig2.png "Title")

# nbody4, runbody4

# nbody5

Unlike other versions, this version does not yet have a **runbody5**, essentially because **nbody5** is not maintained, and **nbody6** should be used.  Yet, the manual page talks about a benchmark and it was fun to compare the performance with the numbers from 1995 when this was documented.

In [ ]:
man nbody5

# nbody6, runbody6

In [ ]:
man runbody6